# Sample solutions for exercises from 4th week of labs

This is only one of possible solutions. There are other possibilities to solve these exercises and how to write some operations.

## Pandas repetition

### Load the dataset
It is downloaded in advance in data directory

In [3]:
%matplotlib inline
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv("data/sales-funnel.csv")
data.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


### 1. Retrieve descriptive statistics about the dataset.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 8 columns):
Account     17 non-null int64
Name        17 non-null object
Rep         17 non-null object
Manager     17 non-null object
Product     17 non-null object
Quantity    17 non-null int64
Price       17 non-null int64
Status      17 non-null object
dtypes: int64(3), object(5)
memory usage: 1.1+ KB


In [6]:
data.describe(include='all')

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
count,17.000000,17,17,17,17,17.000000,17.000000,17
unique,NaN,12,5,2,4,NaN,NaN,4
top,NaN,Trantow-Barrows,Cedric Moss,Debra Henley,CPU,NaN,NaN,presented
freq,NaN,3,4,9,9,NaN,NaN,6
mean,462254.235294,NaN,NaN,NaN,NaN,1.764706,30705.882353,NaN
std,259093.442862,NaN,NaN,NaN,NaN,1.032558,28444.605609,NaN
min,141962.000000,NaN,NaN,NaN,NaN,1.000000,5000.000000,NaN
25%,218895.000000,NaN,NaN,NaN,NaN,1.000000,7000.000000,NaN
50%,412290.000000,NaN,NaN,NaN,NaN,2.000000,30000.000000,NaN
75%,714466.000000,NaN,NaN,NaN,NaN,2.000000,40000.000000,NaN


### 2. How many rows has price higher than $8,000?

In [7]:
len(data[data.Price > 8000])

12

This command selects corresponding rows and counts them. 

As we are not interested in these rows themselves, but only their number, the same result can be obtained in a more efficient way. By counting occurrences where the condition is satisfied. You don't have to retrieve the data and when processing big amounts of data, this can save a lot of time and resources.

In [8]:
sum(data.Price > 8000)

12

Execution of the condition returns a binary vector. True values are considered 1 in the sum operation and False are considered as 0. By summing them, we obtain the number of rows where the condition was satisfied.

### 3. Koľko riadkov má status "pending" a cenu vyšiu ako $8,000?

In [9]:
sum((data.Status == 'pending') & (data.Price > 8000))

1

Pozor na zátvorky potrebné kvôli priorite operátorov. Pozor na operátor & a nie and

### 4. Koľko riadkov má status "pending" alebo cenu vyšiu ako $8,000? 

In [10]:
sum((data.Status == 'pending') | (data.Price > 8000))

15

### 5. Vytvorte stĺpec s názvom "amount", ktorý bude tvorený násobkom stĺpcpv "price" a "quantity". Aký je súčet hodnôt v tomto stĺpci pre riadky so statusom "won"?

In [11]:
data['amount'] = data.Price * data.Quantity
data.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status,amount
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented,30000
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented,10000
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending,10000
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined,35000
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won,130000


In [13]:
data[data.Status == 'won'].amount.sum()

781000

### 6. Aký je celkový súčet stĺpcu "amount" pre záznamy produktov z kategórie "CPU"?

In [14]:
data[data.Product == 'CPU'].amount.sum()

1100000

## Zmena tvaru tabuliek

### Pivot table

### 2. Aký je rozdiel medzi funkciami "pivot" a "pivot_table"?

pivot_table umožňuje definovať agregačnú funkciu, ktorá spája hodnoty pri duplicitných kombináciách v indexe a poemnovaniach stĺpcov.  

### 3. V dátovej sade z predchádzajúcich úloh vytvorte tabuľku, kde v riadkoch budú záznamy pre rôzne produkty, v stĺpcoch záznamy pre rôzne statusy a hodnoty budú agregované hodnoty zo stĺpcu "amount". Na agregáciu použite funkciu sum z balíčka numpy.

In [15]:
pd.pivot_table(data, values='amount', index='Product', columns='Status', aggfunc=np.sum)

Status,declined,pending,presented,won
Product,,,,
CPU,200000.0,80000.0,60000.0,760000.0
Maintenance,NaN,25000.0,NaN,21000.0
Monitor,NaN,NaN,10000.0,NaN
Software,NaN,NaN,30000.0,NaN


### Groupby
### 1. Spravte úlohu 3 zo sekcie pre Pivot table pomocou operácie groupby.

In [19]:
data.groupby(['Product', 'Status']).amount.sum().unstack()

Status,declined,pending,presented,won
Product,,,,
CPU,200000.0,80000.0,60000.0,760000.0
Maintenance,NaN,25000.0,NaN,21000.0
Monitor,NaN,NaN,10000.0,NaN
Software,NaN,NaN,30000.0,NaN


### Cross tab
### 1. Vytvorte kontingenčnú tabuľku (crosstab), ktorá ukáže koľko záznamov je v jednotlivých stavoch (status) pre jednotlivé produkty

In [20]:
pd.crosstab(data.Product, data.Status)

Status,declined,pending,presented,won
Product,,,,
CPU,3,1,2,3
Maintenance,0,3,0,1
Monitor,0,0,1,0
Software,0,0,3,0


### 2. Vytvorte tabuľku z prvej úlohy, ale nahraďte produkty za Manažérov alebo Rep alebo dvojice Manažér a Rep. Vzniknú vám teda 3 rôzne tabuľky.

In [22]:
pd.crosstab(data.Manager, data.Status)

Status,declined,pending,presented,won
Manager,,,,
Debra Henley,2,3,3,1
Fred Anderson,1,1,3,3


In [23]:
pd.crosstab(data.Rep, data.Status)

Status,declined,pending,presented,won
Rep,,,,
Cedric Moss,0,1,2,1
Craig Booker,1,1,2,0
Daniel Hilton,0,1,1,1
John Smith,1,1,0,0
Wendy Yule,1,0,1,2


In [24]:
pd.crosstab([data.Manager, data.Rep], data.Status)

Status                       declined  pending  presented  won
Manager       Rep                                             
Debra Henley  Craig Booker          1        1          2    0
              Daniel Hilton         0        1          1    1
              John Smith            1        1          0    0
Fred Anderson Cedric Moss           0        1          2    1
              Wendy Yule            1        0          1    2

## Iný dataset
### 1. Načítajte dataset. Pri volaní funkcie read_csv použite kódovanie "latin1".

In [67]:
data2 = pd.read_csv('data/crunchbase_monthly_export.csv', encoding='latin1')
data2.head()

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at,Unnamed: 18
0,/organization/canal-do-credito,Canal do Credito,http://www.canaldocredito.com.br,|Credit|Technology|Services|Finance|,Credit,"750,000",NaN,BRA,NaN,Rio de Janeiro,Belo Horizonte,1,NaN,NaN,NaN,NaN,1/1/10,1/1/10,NaN
1,/organization/waywire,#waywire,http://www.waywire.com,|Entertainment|Politics|Social Media|News|,Entertainment,"1,750,000",acquired,USA,NY,New York City,New York,1,6/1/12,2012-06,2012-Q2,2012.0,6/30/12,6/30/12,NaN
2,/organization/tv-communications,&TV Communications,http://enjoyandtv.com,|Games|,Games,"4,000,000",operating,USA,CA,Los Angeles,Los Angeles,2,NaN,NaN,NaN,NaN,6/4/10,9/23/10,NaN
3,/organization/rock-your-paper,'Rock' Your Paper,http://www.rockyourpaper.org,|Publishing|Education|,Education,"40,000",operating,EST,NaN,Tallinn,Tallinn,1,10/26/12,2012-10,2012-Q4,2012.0,8/9/12,8/9/12,NaN
4,/organization/in-touch-network,(In)Touch Network,http://www.InTouchNetwork.com,|Electronics|Guides|Coffee|Restaurants|Music|i...,Apps,"1,500,000",operating,GBR,NaN,London,London,1,4/1/11,2011-04,2011-Q2,2011.0,4/1/11,4/1/11,NaN


### 2. Získajte deskriptívne štatistiky o datasete. O aký dataset ide?

Ide o dataset informácií o firmách/projektoch, ktoré žiadali o investíciu a o tom, či boli v žiadosti úspešní

In [68]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44950 entries, 0 to 44949
Data columns (total 19 columns):
permalink              44950 non-null object
name                   44949 non-null object
homepage_url           41914 non-null object
category_list          40997 non-null object
 market                40997 non-null object
 funding_total_usd     44950 non-null object
status                 42208 non-null object
country_code           40805 non-null object
state_code             27773 non-null object
region                 39615 non-null object
city                   40109 non-null object
funding_rounds         44950 non-null int64
founded_at             35292 non-null object
founded_month          35226 non-null object
founded_quarter        35226 non-null object
founded_year           35226 non-null float64
first_funding_at       44950 non-null object
last_funding_at        44950 non-null object
Unnamed: 18            0 non-null float64
dtypes: float64(2), int64(1), object(16

Dataset má veľa stĺpcov obsahujúcich reťazce aj keď by sme tam logicky očakávali číslo. Obsahuje tiež celkom vela chýbajúcich hodnôt.  

In [69]:
data2.describe(include='all')

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at,Unnamed: 18
count,44950,44949,41914,40997,40997,44950,42208,40805,27773,39615,40109,44950.000000,35292,35226,35226,35226.000000,44950,44950,0.0
unique,44947,44892,41781,13191,547,13558,3,127,177,1282,4829,NaN,3146,398,207,NaN,3757,3502,NaN
top,/organization/henley-putnam-university,Spire,http://valerion.com,|Software|,Biotechnology,-,operating,USA,CA,SF Bay Area,San Francisco,NaN,1/1/11,2011-01,2012-Q1,NaN,1/1/12,1/1/13,NaN
freq,2,3,2,3597,3868,7406,36135,26540,9164,6242,2356,NaN,2026,2146,2632,NaN,445,362,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.681135,NaN,NaN,NaN,2007.083461,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.269198,NaN,NaN,NaN,7.522161,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,1900.000000,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,2005.000000,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,2009.000000,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,2011.000000,NaN,NaN,NaN


### 3. Aký je maximálny získaný funding? Pomôcka: budete musieť transformovať a upraviť hodnoty. Poriadne sa pozrite na dátový typ v stĺpci a na názov stĺpca.

In [70]:
data2.columns

Index(['permalink', 'name', 'homepage_url', 'category_list', ' market ',
       ' funding_total_usd ', 'status', 'country_code', 'state_code', 'region',
       'city', 'funding_rounds', 'founded_at', 'founded_month',
       'founded_quarter', 'founded_year', 'first_funding_at',
       'last_funding_at', 'Unnamed: 18'],
      dtype='object')

niektoré stĺpce majú v názvoch medzery na začiatku a na konci

In [71]:
data2[' funding_total_usd '].head(10)

0       750,000 
1     1,750,000 
2     4,000,000 
3        40,000 
4     1,500,000 
5       600,000 
6     7,000,000 
7     4,912,393 
8     2,000,000 
9           -   
Name:  funding_total_usd , dtype: object

sledovaný stĺpec obsahuje reťazce a viaceré nechcené symboly, ktoré bude treba odstrániť

In [72]:
data2[' funding_total_usd '] = pd.to_numeric(data2[' funding_total_usd '].str.replace(r'\D', ''))

**Pozor**, v predchádzajúcej bunke robím nebezpečnú operáciu. Mením hodnoty toho istého stĺpca, ktorý práve spracovávam. **Po správnosti by som mal vytvoriť nový stĺpec s iným názvom.** Ak totiž táto transformácia spraví niečo neočakávané, tak si nezachovám pôvodnú kópiu dát a nebudem ju vedieť spustiť odznovu. Budem musieť zopakovať všetky kroky od načítana dát. V skutočnosti aj ak by som opakovane pstil túto bunku, tak spadne pretože pri druhom pokuse sa budem snažiť pracovať so stĺpcom už transformovaným na číslo ako kebyt to bol reťazec.  

### 4. Premenujte stĺpce tak aby boli zrozumiteľné a správne formátované.

In [80]:
data2 = data2.rename(columns=str.strip)
data2.columns

Index(['permalink', 'name', 'homepage_url', 'category_list', 'market',
       'funding_total_usd', 'status', 'country_code', 'state_code', 'region',
       'city', 'funding_rounds', 'founded_at', 'founded_month',
       'founded_quarter', 'founded_year', 'first_funding_at',
       'last_funding_at', 'Unnamed: 18'],
      dtype='object')

### 5. Vytvorte tabuľku hodnôt zo stĺpcu "Market" a súčtu hodnôt total funding pre záznamy s danou hodnotou v stĺpci "Market". Usporiadajte tieto hodnoty od najväčšej po menjmenšiu a zobrazte prvých 10 z nich.

In [85]:
data2.groupby('market').funding_total_usd.sum().reset_index().sort_values(by='funding_total_usd', ascending=False).head(10)

,market,funding_total_usd
56,Biotechnology,8.305951e+10
366,Mobile,4.803336e+10
89,Clean Technology,4.325697e+10
501,Software,3.324635e+10
185,E-Commerce,2.527492e+10
8,Advertising,2.011370e+10
209,Enterprise Software,1.935088e+10
266,Health Care,1.880666e+10
146,Curated Web,1.614857e+10
265,Hardware + Software,1.448039e+10


### 6. Koľko spoločností obsahuje kategóriu "Game"?

In [100]:
sum(data2.category_list.apply(lambda x: 'Game' in str(x).split('|')))

57

### 7. Aký je priemerný total funding pre spoločnosti z New Yorku? Ako je to pre spoločnosti zo San Francisca? Porovnajte tieto hodnoty

In [103]:
data2[data2.city == 'New York'].funding_total_usd.mean()

33397114.94209612

In [104]:
data2[data2.city == 'San Francisco'].funding_total_usd.mean()

17075550.37444499